Antes de cualquier cosa, debes autentificarte a través del cmd, ejecutando:<br>
earthengine authenticate <br>
y antes deberías instalar pip install earthengine-api geemap geopandas pandas

In [4]:
import ee
import geopandas as gpd
import pandas as pd
from datetime import datetime
from shapely.geometry import mapping
import json
from tqdm import tqdm

Hay que ir a 
https://console.cloud.google.com/apis/dashboard?invt=AbuKWg&project=maximal-osprey-287122
para habilitar la consulta.

In [5]:
# Inicializar Earth Engine
ee.Initialize(project='maximal-osprey-287122')  # Reemplaza con tu ID real

In [23]:
# Convertir a EE FeatureCollection correctamente
def gdf_to_fc(tabla_geometria):
    features = []
    for _, row in tabla_geometria.iterrows():
        # Convierte la geometría a formato geojson (dict) con mapping()
        geojson_geom = mapping(row['geometry'])
        geom = ee.Geometry(geojson_geom)
        props = row.drop('geometry').to_dict()
        features.append(ee.Feature(geom, props))
    return ee.FeatureCollection(features)

# El NDVI se calcula utilizando las bandas de infrarrojo cercano (B8) y rojo (B4) de Sentinel-2:
def calcular_ndvi(image):
    # Calcular NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return ndvi

In [24]:
# Transformamos el objeto al que pueda consumirse desde Google Earth
polygon = gdf_to_fc(parroquia)

In [26]:
# Cargar shapefile local
shapefile_path = "C:/Users/Michael/Documents/ml_zc/ecu_adm_2024/ecu_adm_adm3_2024.shp"
gdf = gpd.read_file(shapefile_path)

# Convertir el polígono en formato GeoJSON
poligono_geojson = json.loads(gdf.to_json())
polygon = ee.Geometry.Polygon(poligono_geojson['features'][0]['geometry']['coordinates'])

In [27]:
# Seleccionar la colección de imágenes Sentinel-2
image_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(polygon) \
    .filterDate('2023-01-01', '2023-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

def calcular_ndvi(image):
    # Calcular NDVI
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return ndvi

# Aplicar la función para calcular el NDVI a todas las imágenes
ndvi_collection = image_collection.map(calcular_ndvi)

In [28]:
# Calcular la media de NDVI para todo el periodo
ndvi_mean = ndvi_collection.mean()

# Reducir el NDVI por el polígono (media dentro del polígono)
ndvi_zonal = ndvi_mean.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=polygon,
    scale=10,  # Resolución espacial de Sentinel-2
    maxPixels=1e8
)

# Imprimir el resultado
print(ndvi_zonal.getInfo())

{'NDVI': 0.1844364882078893}


In [29]:
# Cargar shapefile local
shapefile_path = "C:/Users/Michael/Documents/ml_zc/ecu_adm_2024/ecu_adm_adm3_2024.shp"
gdf = gpd.read_file(shapefile_path)
parroquia = gdf[gdf['ADM3_ES'] == "Quito"][['ADM3_PCODE', 'geometry']]

# Convertir el polígono en formato GeoJSON
poligono_geojson = json.loads(gdf.to_json())
polygon = ee.Geometry.Polygon(poligono_geojson['features'][0]['geometry']['coordinates'])

# Extraemos el código de la parroquia para usarla luego
name_parr = parroquia['ADM3_PCODE'].tolist()[0]

# Definir años a procesar
years = list(range(2014, 2025))

# Extraer VIIRS diciembre por parroquia
results = []

# Usamos tqdm para la barra de progreso
for year in tqdm(years, desc="Procesando años", ncols=100):
    star = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)
    
    image_collection = ee.ImageCollection('COPERNICUS/S2') \
                    .filterBounds(polygon) \
                    .filterDate(star, end) \
                    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    
    # Verificar cuántas imágenes fueron seleccionadas
    num_images = image_collection.size().getInfo()
    print(f"Imágenes seleccionadas para {year}: {num_images}")
    
    if num_images > 0:
        # Aplicar la función para calcular el NDVI a todas las imágenes
        ndvi_collection = image_collection.map(calcular_ndvi)
    
        # Calcular la media de NDVI para todo el periodo
        ndvi_mean = ndvi_collection.mean()

        # Reducir el NDVI por el polígono (media dentro del polígono)
        ndvi_zonal = ndvi_mean.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=polygon,
            scale=10,  # Resolución espacial de Sentinel-2
            maxPixels=1e8
        )
        
        # Verificar el contenido del resultado
        ndvi_zonal_info = ndvi_zonal.getInfo()
        print(f"Resultado de NDVI para {year}: {ndvi_zonal_info}")
        
        if ndvi_zonal_info:
            results.append({'ADM3_PCODE': name_parr, 'Año': year, 'avg_ndvi': ndvi_zonal_info['NDVI']})
        else:
            print(f"No se obtuvieron valores de NDVI para {year}")
    else:
        print(f"No se encontraron imágenes para el año {year}")

# Exportar resultados a CSV si hay resultados
if results:
    df = pd.DataFrame(results)
    df.to_csv('resultados_ndvi.csv', index=False)
else:
    print("No se generaron resultados de NDVI.")

Procesando años:   9%|████▎                                          | 1/11 [00:02<00:24,  2.41s/it]

Imágenes seleccionadas para 2014: 0
No se encontraron imágenes para el año 2014


Procesando años:  18%|████████▌                                      | 2/11 [00:15<01:17,  8.65s/it]

Imágenes seleccionadas para 2015: 0
No se encontraron imágenes para el año 2015
Imágenes seleccionadas para 2016: 2


Procesando años:  27%|████████████▊                                  | 3/11 [00:38<02:00, 15.08s/it]

...

Resultado de NDVI para 2022: {'NDVI': 0.17217034451486862}
Imágenes seleccionadas para 2023: 5


Procesando años:  91%|█████████████████████████████████████████▊    | 10/11 [02:37<00:21, 21.65s/it]

Resultado de NDVI para 2023: {'NDVI': 0.16609461224145747}
Imágenes seleccionadas para 2024: 3


Procesando años: 100%|██████████████████████████████████████████████| 11/11 [02:50<00:00, 15.51s/it]

Resultado de NDVI para 2024: {'NDVI': 0.13496706625573832}
